In [ ]:
import pandas as pd
import os

files = os.listdir("../raw_data")

df_lst = []
for num, file_name in enumerate(files):
    df = pd.read_excel(f"../raw_data/{file_name}", engine="openpyxl")

    BM = df["주제분류번호"].str.slice(0, 3)
    new_df = df[(BM == "004") | (BM == "005")]

    BM = new_df["등록일자"].str.slice(0, 4)
    new_df[BM.astype(int) >= 2018]

    new_df["도서관"] = num

    df_lst.append(new_df)


In [ ]:
modified_data = pd.concat(df_lst)

In [ ]:
BM = modified_data["등록일자"].str.slice(0, 4)
modified_data = modified_data[BM.astype(int) >= 2018]

modified_data = modified_data.drop(columns=["번호", "발행년도", "세트 ISBN", "부가기호", "권", "도서권수", "대출건수"])
modified_data = modified_data.reset_index(drop=True)


modified_data.to_parquet("raw_data.parquet")

In [ ]:
files = os.listdir("../raw_data")
lib_name = list(map(lambda x: x[8:10], files))
lib_name[0] = "동대문"
lib_name[8] = "영등포"
lib_name[11] = "서대문"

lib_dict = dict(zip(range(20), lib_name))
lib_books = modified_data[["도서관", "ISBN"]]
lib_books.loc[:, "도서관"] = lib_books["도서관"].replace(lib_dict)


lib_books.to_parquet("lib_books.parquet")


In [ ]:
book_info = modified_data.drop(columns="도서관")
book_info.drop_duplicates(subset=["ISBN"], inplace=True)


book_info.to_parquet("book_info.parquet")
